In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [2]:
with open('/Volumes/GoogleDrive/Мой диск/HSE/Polar_Codes/lstm_train_2_L8_130K.pickle', 'rb') as f:
    data = pickle.load(f)
  

In [3]:
len(data)

13

In [4]:
len(data[0])

10000

In [5]:
len(data[0][1][0])

512

In [6]:
X = []
y = []
for i in range(len(data)):
    for j in range(len(data[i])):
        X.append(data[i][j][0])
        y.append(data[i][j][1])

In [7]:
with open('/Volumes/GoogleDrive/Мой диск/HSE/Polar_Codes/lstm_train_3_L8_xK.pickle', 'rb') as f:
    data2 = pickle.load(f)

In [8]:
len(data2)

13

In [9]:
len(data2[0])

20000

In [10]:
for i in range(len(data2)):
    for j in range(len(data2[i])):
        X.append(data2[i][j][0])
        y.append(data2[i][j][1])

In [38]:
X = np.array(X)
y = np.array(y)

In [39]:
vc = pd.Series(y).value_counts()

In [40]:
y = pd.Series(y)

In [41]:
y.value_counts(normalize=True)[:15]

8     0.094726
2     0.089891
1     0.075466
0     0.064966
6     0.048236
13    0.045564
7     0.041655
14    0.037158
10    0.027929
9     0.027757
5     0.025422
3     0.020994
17    0.016937
4     0.016410
40    0.016004
dtype: float64

In [42]:
idx = y.notna() & y.isin(list(vc[:62].index))
y = y[idx]
X = X[idx]

In [43]:
X.mean(), X.std()

(-0.0017556342364824253, 0.9999984588730303)

In [44]:
X = (X - X.mean(axis=0))/X.std(axis=0)

<ipython-input-44-ca814e53643e>:1: RuntimeWarning: invalid value encountered in true_divide
  X = (X - X.mean(axis=0))/X.std(axis=0)


In [45]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(512, 256)
        self.act1 = nn.Hardswish ()
        self.bn1 = nn.BatchNorm1d(256, momentum=.9999)

        self.lin2 = nn.Linear(256, 128)
        self.act2 = nn.Hardswish ()
        self.bn2 = nn.BatchNorm1d(128, momentum=.9999)

        self.lin3 = nn.Linear(128, 64)
        self.act3 = nn.Hardswish()
        self.bn3 = nn.BatchNorm1d(64, momentum=.9999)

        self.lin4 = nn.Linear(64, 62)

    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.bn1(x)
        # x = self.drop1(x)

        x = self.lin2(x)
        x = self.act2(x)
        x = self.bn2(x)
        #x = self.drop2(x)

        x = self.lin3(x)
        x = self.act3(x)
        x = self.bn3(x)
        #x = self.drop3(x)

        x = self.lin4(x)
        return x


In [21]:
class lstm_pol(nn.Module):

    def __init__(self):
        super(lstm_pol, self).__init__()
        self.seq_len = 512
        self.n_hidden = 256
        self.n_layers = 2
        self.lstm = nn.LSTM(1, 256, self.n_layers, dropout=0.1)
        self.act1 = nn.PReLU(256)
        self.lin1 = nn.Linear(256, 62)

    def reset_hidden_state(self):
        self.hidden = (
          torch.zeros(self.n_layers, 512, 256).to(device),
          torch.zeros(self.n_layers, 512, 256).to(device)
      )
    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(
            sequences.view(len(sequences), self.seq_len, -1),
            self.hidden
        )
        x = \
            lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        x = self.act1(x)
        y_pred = self.lin1(x)
        return y_pred

In [20]:
class gru_pol(nn.Module):

    def __init__(self):
        super(gru_pol, self).__init__()
        self.seq_len = 512
        self.n_hidden = 100
        self.n_layers = 1
        self.lstm = nn.GRU(1, self.n_hidden, self.n_layers, dropout=0.1)
        #self.act1 = nn.PReLU(256)
        self.bn1 = nn.BatchNorm1d(self.n_hidden)
        self.lin1 = nn.Linear(self.n_hidden, 62)

    def reset_hidden_state(self):
        
      self.hidden = torch.zeros(self.n_layers, 512, self.n_hidden).to(device)

    def forward(self, sequences):
      self.reset_hidden_state()
      lstm_out, self.hidden = self.lstm(
        sequences.view(len(sequences), self.seq_len, -1),
        self.hidden
      )
      x = \
        lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
      #x = self.act1(x)
      x = self.bn1(x)
      y_pred = self.lin1(x)
      return y_pred

In [46]:
X.shape

(174767, 512)

In [47]:
pd.factorize(y)

(array([ 0,  1,  1, ..., 61, 60, 57]),
 Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
             51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61],
            dtype='int64'))

In [48]:
y = pd.factorize(y)[0]

In [49]:
y.max()

61

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/6, random_state=42)

In [51]:
device = torch.device(0)

In [52]:
X_train= torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.tensor(y_train.astype(int), dtype=int)
y_test = torch.tensor(y_test.astype(int), dtype=int)

In [53]:
dataset = TensorDataset(X_train, y_train)

In [54]:
loader = DataLoader(dataset, batch_size=200, shuffle=True, drop_last=True)

In [55]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=True)

In [ ]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, nesterov=False, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.5)

In [ ]:
# Линейная
epochs = 48
best_epochs_loss=np.inf
best_epoch_num = 0
for i in range(epochs):
    loss_tr = 0
    qq = 0
    train_res = []
    model.train()
    # model.bn1.momentum = 1e-5
    # model.bn2.momentum = 1e-5
    for x_tr, y_tr in loader:
        x_tr = x_tr.to(device)
        y_tr = y_tr.to(device)
        optimizer.zero_grad()
        #x_tr = x_tr.permute(1, 0)
        #x_tr = x_tr.unsqueeze(2)
        y_pred = model(x_tr)
        y_pred = y_pred#[-1, :, :]
        single_loss = criterion(y_pred, y_tr)
        single_loss.backward()
        loss_tr += single_loss.item()
        qq += 1
        optimizer.step()
        y_tr = y_tr.cpu().numpy()
        y_pred = y_pred.detach().cpu().numpy()
        y_pred = np.argsort(y_pred)[:, -10:]
        y_tr = np.array([y_tr])
        train_res += list(np.any((y_pred - y_tr.T) == 0, axis=1))
    print(f'epoch: {i:3} loss: {loss_tr/qq:10.8f}')
    qq = 0
    with torch.no_grad():
      model.eval()
      # model.bn1.train()
      # model.bn2.train()
      # model.bn1.momentum = 0
      # model.bn2.momentum = 0
      q = 0
      loss_test = 0
      test_res = []
      for x_te, y_te in test_loader:
        # x_te = x_te.permute(1, 0)
        # x_te = x_te.unsqueeze(2)
        x_te = x_te.to(device)
        y_te = y_te.to(device)
        pred_test = model(x_te)
        pred_test = pred_test#[-1, :, :]
        loss_test += criterion(pred_test, y_te).item()
        y_te = y_te.cpu().numpy()
        pred_test = pred_test.cpu().numpy()
        pred_test = np.argsort(pred_test)[:, -10:]
        y_te = np.array([y_te])
        test_res += list(np.any((pred_test - y_te.T) == 0, axis=1))
        q += 1
      loss_test /= q
    print(f'epoch: {i:3} loss: {loss_test:10.8f} loss_test: {loss_test:10.8f}')
    q = 0
    test_res =np.mean( np.array(test_res))
    train_res =np.mean(np.array(train_res))
    print(f'epoch: {i} acc_train: {train_res}, acc_test: {test_res}')
    if loss_test < best_epochs_loss:
      best_epochs_loss = loss_test
      best_epoch_num = 0
    else:
      if best_epoch_num >= 5:
        scheduler.step()
        print('step')
        best_epoch_num = 0
      else:
        best_epoch_num += 1

epoch:   0 loss: 3.09011760
epoch:   0 loss: 3.00899721 loss_test: 3.00899721
epoch: 0 acc_train: 0.6900274725274725, acc_test: 0.7089398516890965
epoch:   1 loss: 2.99215606
epoch:   1 loss: 2.99259580 loss_test: 2.99259580
epoch: 1 acc_train: 0.7163598901098901, acc_test: 0.7164583905520462
epoch:   2 loss: 2.97858611
epoch:   2 loss: 2.98084273 loss_test: 2.98084273
epoch: 2 acc_train: 0.7209821428571429, acc_test: 0.7205781378742104
epoch:   3 loss: 2.97073861
epoch:   3 loss: 2.99011092 loss_test: 2.99011092
epoch: 3 acc_train: 0.723798076923077, acc_test: 0.7163553968689921
epoch:   4 loss: 2.96478286
epoch:   4 loss: 2.98949099 loss_test: 2.98949099
epoch: 4 acc_train: 0.7266689560439561, acc_test: 0.7183809393023894
epoch:   5 loss: 2.95988415
epoch:   5 loss: 2.96326508 loss_test: 2.96326508
epoch: 5 acc_train: 0.7272252747252748, acc_test: 0.7258308157099698
epoch:   6 loss: 2.95709159
epoch:   6 loss: 2.96357005 loss_test: 2.96357005
epoch: 6 acc_train: 0.7289148351648351, a

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'drive/My Drive/polar_linear_model.pth')


In [56]:
model = lstm_pol().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, 0.2)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# lstm
epochs = 10000
for i in range(epochs):
    #model.reset_hidden_state()
    loss_tr = 0
    qq = 0
    train_res = []
    for x_tr, y_tr in loader:
        x_tr = x_tr.to(device)
        y_tr = y_tr.to(device)
        optimizer.zero_grad()
        #x_tr = x_tr.permute(1, 0)
        #x_tr = x_tr.unsqueeze(2)
        y_pred = model(x_tr)
        y_pred = y_pred#[-1, :, :]
        single_loss = criterion(y_pred, y_tr)
        single_loss.backward()
        loss_tr += single_loss.item()
        qq += 1
        optimizer.step()
        y_tr = y_tr.cpu().numpy()
        y_pred = y_pred.detach().cpu().numpy()
        y_pred = np.argsort(y_pred)[:, -10:]
        y_tr = np.array([y_tr])
        train_res += list(np.any((y_pred - y_tr.T) == 0, axis=1))
        for hid in model.hidden:
          hid.detach_()
    print(f'epoch: {i:3} loss: {loss_tr/qq:10.8f}')
    qq = 0
    scheduler.step()
    with torch.no_grad():
      q = 0
      loss_test = 0
      test_res = []
      for x_te, y_te in test_loader:
        # x_te = x_te.permute(1, 0)
        # x_te = x_te.unsqueeze(2)
        x_te = x_te.to(device)
        y_te = y_te.to(device)
        pred_test = model(x_te)
        pred_test = pred_test#[-1, :, :]
        loss_test += criterion(pred_test, y_te).item()
        y_te = y_te.cpu().numpy()
        pred_test = pred_test.cpu().numpy()
        pred_test = np.argsort(pred_test)[:, -10:]
        y_te = np.array([y_te])
        test_res += list(np.any((pred_test - y_te.T) == 0, axis=1))
        q += 1
      loss_test /= q
    print(f'epoch: {i:3} loss: {loss_test:10.8f} loss_test: {loss_test:10.8f}')
    q = 0
    test_res =np.mean( np.array(test_res))
    train_res =np.mean( np.array(train_res))
    print(f'epoch: {i} acc_train: {train_res}, acc_test: {test_res}')

epoch:   0 loss: 3.62151206
epoch:   0 loss: 3.61895375 loss_test: 3.61895375
epoch: 0 acc_train: 0.5505494505494506, acc_test: 0.5491966492721779
epoch:   1 loss: 3.61135822
epoch:   1 loss: 3.62069205 loss_test: 3.62069205
epoch: 1 acc_train: 0.5532829670329671, acc_test: 0.5451455644053831
epoch:   2 loss: 3.61028657
epoch:   2 loss: 3.62079322 loss_test: 3.62079322
epoch: 2 acc_train: 0.5536813186813186, acc_test: 0.5491966492721779


KeyboardInterrupt: ignored

In [ ]:
model = gru_pol().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, 0.2)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
# gru
epochs = 10000
for i in range(epochs):
    #model.reset_hidden_state()
    loss_tr = 0
    qq = 0
    train_res = []
    for x_tr, y_tr in loader:
        x_tr = x_tr.to(device)
        y_tr = y_tr.to(device)
        optimizer.zero_grad()
        #x_tr = x_tr.permute(1, 0)
        #x_tr = x_tr.unsqueeze(2)
        y_pred = model(x_tr)
        y_pred = y_pred#[-1, :, :]
        single_loss = criterion(y_pred, y_tr)
        single_loss.backward()
        loss_tr += single_loss.item()
        qq += 1
        optimizer.step()
        y_tr = y_tr.cpu().numpy()
        y_pred = y_pred.detach().cpu().numpy()
        y_pred = np.argsort(y_pred)[:, -10:]
        y_tr = np.array([y_tr])
        train_res += list(np.any((y_pred - y_tr.T) == 0, axis=1))
    print(f'epoch: {i:3} loss: {loss_tr/qq:10.8f}')
    qq = 0
    scheduler.step()
    with torch.no_grad():
      q = 0
      loss_test = 0
      test_res = []
      for x_te, y_te in test_loader:
        # x_te = x_te.permute(1, 0)
        # x_te = x_te.unsqueeze(2)
        x_te = x_te.to(device)
        y_te = y_te.to(device)
        pred_test = model(x_te)
        pred_test = pred_test#[-1, :, :]
        loss_test += criterion(pred_test, y_te).item()
        y_te = y_te.cpu().numpy()
        pred_test = pred_test.cpu().numpy()
        pred_test = np.argsort(pred_test)[:, -10:]
        y_te = np.array([y_te])
        test_res += list(np.any((pred_test - y_te.T) == 0, axis=1))
        q += 1
      loss_test /= q
    print(f'epoch: {i:3} loss: {loss_test:10.8f} loss_test: {loss_test:10.8f}')
    q = 0
    test_res =np.mean(np.array(test_res))
    train_res =np.mean(np.array(train_res))
    print(f'epoch: {i} acc_train: {train_res}, acc_test: {test_res}')

epoch:   0 loss: 4.01502958
epoch:   0 loss: 3.88894039 loss_test: 3.88894039
epoch: 0 acc_train: 0.4503434065934066, acc_test: 0.5377643504531722
epoch:   1 loss: 3.77103266
epoch:   1 loss: 3.68909415 loss_test: 3.68909415
epoch: 1 acc_train: 0.5509684065934066, acc_test: 0.5479607250755287
epoch:   2 loss: 3.65133192
epoch:   2 loss: 3.64125207 loss_test: 3.64125207
epoch: 2 acc_train: 0.5536744505494505, acc_test: 0.5483383685800605
epoch:   3 loss: 3.62506145
epoch:   3 loss: 3.62871083 loss_test: 3.62871083
epoch: 3 acc_train: 0.5542239010989011, acc_test: 0.5487160120845922
epoch:   4 loss: 3.61667750
epoch:   4 loss: 3.62338086 loss_test: 3.62338086
epoch: 4 acc_train: 0.554010989010989, acc_test: 0.5491623180444932
epoch:   5 loss: 3.61271428
epoch:   5 loss: 3.62068721 loss_test: 3.62068721
epoch: 5 acc_train: 0.5537980769230769, acc_test: 0.5493339741829167
epoch:   6 loss: 3.61077107
epoch:   6 loss: 3.61898774 loss_test: 3.61898774
epoch: 6 acc_train: 0.554059065934066, ac

KeyboardInterrupt: ignored

In [ ]:
nn.BatchNorm2d??

In [ ]:
class conv(nn.Module):

    def __init__(self):
        super(conv, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 8)
        self.act1 = nn.Hardswish()
        self.pool1 = nn.MaxPool1d(8, 8)
        self.bn1 = nn.BatchNorm1d(16, momentum=0.9999)

        self.conv2 = nn.Conv1d(16, 32, 8)
        self.act2 = nn.Hardswish()
        self.pool2 = nn.MaxPool1d(8, 8)
        self.bn2 = nn.BatchNorm1d(32, momentum=0.9999)


        self.lin1 = nn.Linear(224, 128)
        self.act3 = nn.Hardswish()
        self.bn3 = nn.BatchNorm1d(128, momentum=0.9999)
        
        self.lin2 = nn.Linear(128, 62)



    def forward(self, x):
      x  = self.conv1(x)
      x = self.act1(x)
      x = self.pool1(x)
      x = self.bn1(x)

      x = self.conv2(x)
      x = self.act2(x)
      x = self.pool2(x)
      x = self.bn2(x)

      x = x.view(-1, 224)

      x = self.lin1(x)
      x = self.act3(x)
      x = self.bn3(x)

      x = self.lin2(x)
      return x

In [ ]:
model = conv().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.5)

In [ ]:
# conv
epochs = 60
best_epochs_loss = np.inf
best_epoch_num = 0
for i in range(epochs):
    loss_tr = 0
    qq = 0
    train_res = []
    for x_tr, y_tr in loader:
        x_tr = x_tr.unsqueeze(1)
        x_tr = x_tr.to(device)
        y_tr = y_tr.to(device)
        optimizer.zero_grad()
        #x_tr = x_tr.permute(1, 0)
        #x_tr = x_tr.unsqueeze(2)
        y_pred = model(x_tr)
        y_pred = y_pred#[-1, :, :]
        single_loss = criterion(y_pred, y_tr)
        single_loss.backward()
        loss_tr += single_loss.item()
        qq += 1
        optimizer.step()
        y_tr = y_tr.cpu().numpy()
        y_pred = y_pred.detach().cpu().numpy()
        y_pred = np.argsort(y_pred)[:, -10:]
        y_tr = np.array([y_tr])
        train_res += list(np.any((y_pred - y_tr.T) == 0, axis=1))
    print(f'epoch: {i:3} loss: {loss_tr/qq:10.8f}')
    qq = 0
    # scheduler.step()
    with torch.no_grad():
      q = 0
      loss_test = 0
      test_res = []
      for x_te, y_te in test_loader:
        # x_te = x_te.permute(1, 0)
        # x_te = x_te.unsqueeze(2)
        x_te = x_te.unsqueeze(1)
        x_te = x_te.to(device)
        y_te = y_te.to(device)
        pred_test = model(x_te)
        pred_test = pred_test#[-1, :, :]
        loss_test += criterion(pred_test, y_te).item()
        y_te = y_te.cpu().numpy()
        pred_test = pred_test.cpu().numpy()
        pred_test = np.argsort(pred_test)[:, -10:]
        y_te = np.array([y_te])
        test_res += list(np.any((pred_test - y_te.T) == 0, axis=1))
        q += 1
      loss_test /= q
    print(f'epoch: {i:3} loss: {loss_test:10.8f} loss_test: {loss_test:10.8f}')
    q = 0
    test_res =np.mean( np.array(test_res))
    train_res =np.mean( np.array(train_res))
    print(f'epoch: {i} acc_train: {train_res}, acc_test: {test_res}')
    if loss_test < best_epochs_loss:
      best_epochs_loss = loss_test
      best_epoch_num = 0
    else:
      if best_epoch_num >= 5:
        scheduler.step()
        print('step')
        best_epoch_num = 0
      else:
        best_epoch_num += 1

epoch:   0 loss: 3.09269282
epoch:   0 loss: 3.00622051 loss_test: 3.00622051
epoch: 0 acc_train: 0.6871497252747253, acc_test: 0.7083218895907718
epoch:   1 loss: 2.98907033
epoch:   1 loss: 2.98176904 loss_test: 2.98176904
epoch: 1 acc_train: 0.7163255494505495, acc_test: 0.7171793463334248
epoch:   2 loss: 2.96598197
epoch:   2 loss: 2.96719607 loss_test: 2.96719607
epoch: 2 acc_train: 0.7246497252747253, acc_test: 0.7241485855534194
epoch:   3 loss: 2.95214660
epoch:   3 loss: 2.96121370 loss_test: 2.96121370
epoch: 3 acc_train: 0.7299656593406594, acc_test: 0.7238052732765724
epoch:   4 loss: 2.94497491
epoch:   4 loss: 2.96126546 loss_test: 2.96126546
epoch: 4 acc_train: 0.7324038461538461, acc_test: 0.7266204339467179
epoch:   5 loss: 2.93559395
epoch:   5 loss: 2.95553603 loss_test: 2.95553603
epoch: 5 acc_train: 0.7355288461538462, acc_test: 0.7328687173853337
epoch:   6 loss: 2.93018174
epoch:   6 loss: 2.94242867 loss_test: 2.94242867
epoch: 6 acc_train: 0.7373832417582418, 

In [ ]:
torch.save(model.state_dict(), 'drive/My Drive/polar_conv.pth')


Int64Index([117, 118, 111, 121, 122, 158, 171, 173, 124, 174, 179, 119, 181,
            159, 279, 283, 225, 199, 226, 212],
           dtype='int64')